In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots

In [2]:
data=pd.read_excel('CW_Data.xlsx',sheet_name='Sheet2')
data

Gender  Programme  Grade  Total  MCQ  Q1  Q2  Q3    Q4  Q5
0         1          3      3   45.0   21   8   4   2  10.0   0
1         2          3      3   43.0   21   4   2   8   8.0   0
2         1          4      2   26.0   24   0   0   2   0.0   0
3         2          1      3   30.0   24   4   0   0   0.0   2
4         1          2      3   27.0   21   0   2   4   0.0   0
..      ...        ...    ...    ...  ...  ..  ..  ..   ...  ..
614       2          1      2   55.0   33   6   4   4   8.0   0
615       2          4      2   41.0   27   6   4   4   0.0   0
616       2          1      2   73.0   36   8   8  11  10.0   0
617       2          1      2   77.0   42   8   8   6  10.0   3
618       2          1      2   22.0   12   4   0   4   2.0   0

[619 rows x 10 columns]

# task 1

In [12]:
from sklearn import model_selection
predictors = ['Gender','Grade','Total','MCQ','Q1','Q2','Q3','Q4','Q5']

x_train, x_test, y_train, y_test = model_selection.train_test_split(data[predictors], data['Programme'], test_size = 0.25, random_state = 1234)

from sklearn.model_selection import GridSearchCV
from sklearn import tree

max_depth=[2,3, 4, 5, 6, 7, 8, 9, 10]
min_samples_split=[2, 4, 6, 8, 10]
min_samples_leaf=[ 3, 4, 5, 6, 7, 8, 9, 10]

parameters = {'max_depth':max_depth, 'min_samples_split':min_samples_split, 'min_samples_leaf':min_samples_leaf}

grid_dtcateg = GridSearchCV(estimator=tree.DecisionTreeClassifier(), param_grid=parameters, cv=10)
grid_dtcateg.fit(x_train, y_train)

print(grid_dtcateg.best_params_)


KeyboardInterrupt: 

In [13]:

from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn import tree
CART_Classifier = tree.DecisionTreeClassifier(max_depth=2, min_samples_leaf=3, min_samples_split=2)
decision_tree = CART_Classifier.fit(x_train, y_train)
pred=CART_Classifier.predict(x_test)
print('模型在测试集上的预测准确率为：\n',metrics.accuracy_score(y_test, pred))
print('模型在训练集上的预测准确率为：\n',metrics.accuracy_score(y_train, CART_Classifier.predict(x_train)))

    
from sklearn.tree import export_graphviz
from IPython.display import Image
import pydotplus
from six import StringIO

# 绘制决策树
dot_data = StringIO()
export_graphviz(
    decision_tree,
    out_file=dot_data,  
    feature_names=predictors,
    class_names=['1','2','3','4'],  
    filled=True,  # 添加filled=True来为节点添加颜色
    rounded=True,  
    special_characters=True
)
# 决策树展现
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png()) 
# 导出决策树图像为svg
graph.write_svg('Decision tree_2_3_2.svg')


模型在测试集上的预测准确率为：
 0.5741935483870968
模型在训练集上的预测准确率为：
 0.6293103448275862


True

# task 2

In [21]:
from sklearn import svm
import pandas as pd
from sklearn import model_selection
from sklearn import metrics

letters=pd.read_excel('CW_Data.xlsx',sheet_name='Sheet2')
predictors = ['Gender','Grade','Total','MCQ','Q1','Q2','Q3','Q4','Q5']
X_train, X_test, y_train, y_test = model_selection.train_test_split(letters[predictors], letters['Programme'],test_size=0.25, random_state=1234)


In [ ]:

'''使用网格搜索法，选择线性可分SVM“类”中的最佳C值'''
C = [0.05, 0.1, 0.5, 1, 2, 5]
parameters = {'C': C}
grid_linear_svc = model_selection.GridSearchCV(estimator=svm.LinearSVC(), param_grid=parameters, scoring='accuracy',
                                               cv=5, verbose=1)
# 模型在训练数据集上的拟合
grid_linear_svc.fit(X_train, y_train)
# 返回交叉验证后的最佳参数值
print(grid_linear_svc.best_params_)
print(grid_linear_svc.best_score_)

# 模型在测试集上的预测
pred_linear_svc = grid_linear_svc.predict(X_test)
# 模型的预测准确率
print('预测准确率：',metrics.accuracy_score(y_test, pred_linear_svc))

In [24]:
'''使用网格搜索法，选择非线性SVM“类”中的最佳C值'''
kernel = ['rbf', 'linear', 'poly', 'sigmoid']
#以上四种为非线性SVM的几种常用的SVM核函数：[径向基核函数(高斯核函数),线性核函数,多项式核函数,,Sigmoid核函数]
C = [0.1, 0.5, 1, 2, 5]
parameters = {'kernel': kernel, 'C': C}
grid_svc = model_selection.GridSearchCV(estimator=svm.SVC(), param_grid=parameters, scoring='accuracy', cv=5, verbose=1)
# 模型在训练数据集上的拟合
grid_svc.fit(X_train, y_train)
# 返回交叉验证后的最佳参数值
print(grid_svc.best_params_, grid_svc.best_score_)

# 模型在测试集上的预测
pred_svc = grid_svc.predict(X_test)
# 模型的预测准确率
print('预测准确率',metrics.accuracy_score(y_test, pred_svc))

Fitting 5 folds for each of 20 candidates, totalling 100 fits
{'C': 0.5, 'kernel': 'linear'} 0.6357643758765779
预测准确率 0.5612903225806452


如上结果所示，经过5重交叉验证后，发现最佳的惩罚系数C为0.5，
最佳的核函数为线性核函数linear。其SVM表现了极佳的效果，模型在训练数据集上的平均准确率高达
63.58%，而且其在测试数据集的预测准确率为56.13%，

# task 3

In [9]:
# 导入第三方包
import pandas as pd

# 读取数据
mushrooms = pd.read_excel('CW_Data.xlsx',sheet_name='Sheet2')


from sklearn import model_selection

# 将数据集拆分为训练集合测试集
Predictors =  ['Gender','Grade','Total','MCQ','Q1','Q2','Q3','Q4','Q5']
X_train, X_test, y_train, y_test = model_selection.train_test_split(mushrooms[Predictors], mushrooms['Programme'],
                                                                    test_size=0.25, random_state=10)

from sklearn import naive_bayes
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt

# 构建多项式贝叶斯分类器的“类”
mnb = naive_bayes.MultinomialNB()
# 基于训练数据集的拟合
mnb.fit(X_train, y_train)
# 基于测试数据集的预测
mnb_pred = mnb.predict(X_test)
# 构建混淆矩阵
cm = pd.crosstab(mnb_pred, y_test)
# 绘制混淆矩阵图


# 模型的预测准确率
print('模型的准确率为：\n', metrics.accuracy_score(y_test, mnb_pred))
#print('模型的评估报告：\n', metrics.classification_report(y_test, mnb_pred))




模型的准确率为：
 0.4838709677419355


In [10]:
import pandas as pd

# 读入评论数据
from sklearn import model_selection
from sklearn import naive_bayes
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
mushrooms = pd.read_excel('CW_Data.xlsx',sheet_name='Sheet2')


from sklearn import model_selection

# 将数据集拆分为训练集合测试集
Predictors =  ['Gender','Grade','Total','MCQ','Q1','Q2','Q3','Q4','Q5']
X_train, X_test, y_train, y_test = model_selection.train_test_split(mushrooms[Predictors], mushrooms['Programme'],
                                                                    test_size=0.25, random_state=10)
# 构建伯努利贝叶斯分类器
bnb = naive_bayes.BernoulliNB()
# 模型在训练数据集上的拟合
bnb.fit(X_train, y_train)
# 模型在测试数据集上的预测
bnb_pred = bnb.predict(X_test)
# 构建混淆矩阵
cm = pd.crosstab(bnb_pred, y_test)

# 模型的预测准确率
print('模型的准确率为：\n', metrics.accuracy_score(y_test, bnb_pred))




模型的准确率为：
 0.5032258064516129


# task 4

In [3]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score
predictors = ['Gender','Grade','Total','MCQ','Q1','Q2','Q3','Q4','Q5']

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(data[predictors], data['Programme'], test_size = 0.25, random_state = 1234)
#采用2，3，2的决策树
tree = DecisionTreeClassifier(criterion='gini', max_depth=2, min_samples_split=3, min_samples_leaf=2)

ada = AdaBoostClassifier(base_estimator = tree, #指定基分类器
                         algorithm='SAMME',#这里还可以指定其他分类器，如感知器、支持向量机、逻辑斯蒂回归等
                        n_estimators = 500,    #子分类器个数
                        learning_rate=0.1,
                        random_state = 1) #random_state参数指定分类器训练时使用的随机算法的随机种子
ada = ada.fit(X_train, Y_train)
Y_train_pred = ada.predict(X_train)
Y_test_pred = ada.predict(X_test)
accuracy_score(Y_train, Y_train_pred)  #得到训练集上准确率
accuracy_score(Y_test, Y_test_pred)    #得到测试集上准确率

D:\APP\Miniconda\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.5612903225806452

In [5]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn import svm
predictors = ['Gender','Grade','Total','MCQ','Q1','Q2','Q3','Q4','Q5']

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(data[predictors], data['Programme'], test_size = 0.25, random_state = 1234)
tree = svm.SVC(kernel='linear', C=0.5)
ada = AdaBoostClassifier(base_estimator = tree, #指定基分类器
                         algorithm='SAMME',#这里还可以指定其他分类器，如感知器、支持向量机、逻辑斯蒂回归等
                        n_estimators = 500,    #子分类器个数
                        learning_rate=0.1,
                        random_state = 1) #random_state参数指定分类器训练时使用的随机算法的随机种子
ada = ada.fit(X_train, Y_train)
Y_train_pred = ada.predict(X_train)
Y_test_pred = ada.predict(X_test)
accuracy_score(Y_train, Y_train_pred)  #得到训练集上准确率
accuracy_score(Y_test, Y_test_pred)    #得到测试集上准确率

D:\APP\Miniconda\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.47096774193548385

In [8]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn import naive_bayes
predictors = ['Gender','Grade','Total','MCQ','Q1','Q2','Q3','Q4','Q5']

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(data[predictors], data['Programme'], test_size = 0.25, random_state = 1234)
tree = naive_bayes.BernoulliNB()
ada = AdaBoostClassifier(base_estimator = tree, #指定基分类器
                         algorithm='SAMME',#这里还可以指定其他分类器，如感知器、支持向量机、逻辑斯蒂回归等
                        n_estimators = 500,    #子分类器个数
                        learning_rate=0.1,
                        random_state = 1) #random_state参数指定分类器训练时使用的随机算法的随机种子
ada = ada.fit(X_train, Y_train)
Y_train_pred = ada.predict(X_train)
Y_test_pred = ada.predict(X_test)
accuracy_score(Y_train, Y_train_pred)  #得到训练集上准确率
accuracy_score(Y_test, Y_test_pred)    #得到测试集上准确率

D:\APP\Miniconda\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.5290322580645161